In [1]:
import pandas as pd
import requests
import json

In [2]:
meteoriteFile = "Resources/meteorite-landings.csv"

In [3]:
meteoriteDF = pd.read_csv(meteoriteFile, encoding="ISO-8859-1")

In [4]:
meteoriteDF.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [5]:
meteoriteFellDF = meteoriteDF[meteoriteDF["fall"] == "Fell"]

In [6]:
meteoriteFellDF = meteoriteFellDF.drop(meteoriteFellDF[(meteoriteFellDF["reclat"] == 0) & (meteoriteFellDF["reclong"] == 0)].index)

In [7]:
meteoriteFellDF = meteoriteFellDF.drop(meteoriteFellDF[(meteoriteFellDF["year"] >= 2016.0) | (meteoriteFellDF["year"] < 860.0)].index)

In [8]:
locationDF = meteoriteFellDF[['id', 'reclat', 'reclong']]

In [9]:
locationDF["city"] = ""
locationDF["country"] = ""

/Users/mario/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/mario/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
locationDF.head()

,id,reclat,reclong,city,country
0,1,50.77500,6.08333,,
1,2,56.18333,10.23333,,
2,6,54.21667,-113.00000,,
3,10,16.88333,-99.90000,,
4,370,-33.16667,-64.95000,,


In [11]:
params = {"key": "xxx"}

In [13]:
for index,row in locationDF.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    lat = row['reclat']
    lng = row['reclong']
    
    params['latlng'] = f"{lat},{lng}"
    
    meteoriteCityCountry = requests.get(base_url,params=params)
    
    meteoriteCityCountry = meteoriteCityCountry.json()
    
    locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][2]["long_name"]
    locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][5]["long_name"]

/Users/mario/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


IndexError: list index out of range

In [14]:
locationDF.head()

,id,reclat,reclong,city,country
0,1,50.77500,6.08333,Aachen,Germany
1,2,56.18333,10.23333,Risskov,
2,6,54.21667,-113.00000,,
3,10,16.88333,-99.90000,,
4,370,-33.16667,-64.95000,,


In [17]:
#params['latlng'] = "50.77500,6.08333"
    
#response = requests.get(base_url, params=params)
#locationData = response.json()
#print(json.dumps(locationData, indent=4, sort_keys=True))

{
    "plus_code": {
        "compound_code": "Q3GM+28 Aachen, Germany",
        "global_code": "9F28Q3GM+28"
    },
    "results": [
        {
            "address_components": [
                {
                    "long_name": "1",
                    "short_name": "1",
                    "types": [
                        "street_number"
                    ]
                },
                {
                    "long_name": "Johannes-Paul-II.-Stra\u00dfe",
                    "short_name": "Johannes-Paul-II.-Stra\u00dfe",
                    "types": [
                        "route"
                    ]
                },
                {
                    "long_name": "Aachen",
                    "short_name": "AC",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "K\u00f6ln",
                    "short_name": "K",
          